In [1]:
import tensorflow as tf
import numpy as np
import scipy as sc
import h5py
import time
import matplotlib.pyplot as plt
import math
import seaborn as sns

In [2]:
input_vector_length = 2
hidden_layer_nodes = 2
output_vector_length = 2
learning_rate = 1e-3

In [3]:
#Initialize placeholders , which are variable shape empty objects into which any size tensor can be inputed
Input_layer = tf.placeholder(tf.float32, [None,input_vector_length],name = 'input')
#Output_layer = tf.placeholder(tf.float32, [None,output_vector_length],name = 'output')
Output_vectors = tf.placeholder(tf.float32, [None,output_vector_length],name = 'labels')

In [4]:
#Weights for the hidden layer and biases, Biases not needed for this particular problem
# for input to hidden layer
W1 = tf.Variable(tf.random_normal([input_vector_length,hidden_layer_nodes],stddev=0.1), name='W1')
# for hidden to output layer
W2 = tf.Variable(tf.random_normal([hidden_layer_nodes,output_vector_length],stddev=0.1), name='W2')
#The problem here was I used MSE in a terrible way
#hidden_layer_1 = tf.contrib.layers.fully_connected(Input_layer, output_vector_length, None,biases_regularizer=)

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
#Create the operations on the hidden layer
hidden_layer_1 = tf.matmul(Input_layer,W1)

In [6]:
#Create operations for the output layer
Output_layer = tf.matmul(hidden_layer_1,W2)

In [7]:
#Create Loss function
mse = tf.reduce_mean(tf.square(Output_layer - Output_vectors))

In [8]:
#The Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate)
grads_and_vars = optimizer.compute_gradients(mse)
optimizer_real = optimizer.apply_gradients(grads_and_vars,)

In [9]:
#Setup initialization
init_op = tf.global_variables_initializer()

In [10]:
#Code to find angle b/w two vectors
def dotproduct(v1, v2):
  return sum((a*b) for a, b in zip(v1, v2))
def length(v):
  return math.sqrt(dotproduct(v, v))
def angle(v1, v2):
  return math.acos(dotproduct(v1, v2) / (length(v1) * length(v2)))


In [11]:
from scipy.linalg import orth
from scipy.linalg import block_diag
training_points = input_vector_length

Gamma = 0 #Parameter to deviate orthogonal vectors slightly
#input_vectors = 10*(np.random.rand(input_vector_length,training_points) - 0.5) #Generate required length of input vectors
#input_vectors = orth(input_vectors) + Gamma*np.random.randn(input_vector_length,training_points) #Orthogonalise them and add some random noise
input_vectors = np.array([[10,0],[10,0.05]]) #If you want to initalize your own training set
#W = np.array([[0.20774353,1.0305219],[-1.2163291,-0.1880631]])
#orthonormal_vectors = np.matmul(W,orthonormal_vectors)
print(input_vectors)

[[10.    0.  ]
 [10.    0.05]]


In [12]:
#Ax = y, We need to invert A => x_train is actually the output of the NN while y_train is the input

A = 10*np.random.randn(input_vector_length,input_vector_length) #Generate a random matrix to invert
x_train = np.matmul(np.linalg.inv(A),input_vectors) # use the generated A and input_vectors to generate x (See Ax = y)
x_validate = np.random.randn(output_vector_length) # Generate a dingle vector for validation
x_train = np.transpose(x_train) #Transpose this for right multiplication
y_train = input_vectors #Set y_train(The one we'll be sending to the feed forward NN) as input_vectors that were initially generated
y_train = np.transpose(y_train) #Keep x_train and y_train consistent
y_validate = np.reshape((np.matmul(A,x_validate)),(1,output_vector_length))#Appropraite shape for the NN
x_validate = np.reshape(x_validate,(1,output_vector_length))#Appropriate shape for the NN

In [13]:
print(x_train)
print(np.shape(x_train))
print(y_train)
print(np.shape(y_train))
print(x_validate)
print(np.shape(x_validate))
print(y_validate)
print(np.shape(y_validate))

[[-1.60263042 -2.29449359]
 [-0.00564609 -0.00363081]]
(2, 2)
[[10.   10.  ]
 [ 0.    0.05]]
(2, 2)
[[-0.62325059  0.24469983]]
(1, 2)
[[-5.10716556  7.66044192]]
(1, 2)


In [17]:
#Some parameters
epochs = 100000
batch_size = 2
alpha = tf.constant(0.9)


In [ ]:
with tf.Session() as sess: #Start the session
    #Intialise the variables
    sess.run(tf.global_variables_initializer())
    total_batch = int(training_points/batch_size) # This is used to calculate the average loss in each iteration
    print(total_batch)
    for epoch in range(epochs):
        average_loss = 0 #initialize average_loss as zero
        for i in range(total_batch):
            #x_batch, y_batch = y_train, x_train
            _,c,gradients_and_vars,w1,w2 = sess.run([optimizer_real,mse,grads_and_vars,W1,W2],feed_dict = {Input_layer: y_train, Output_vectors: x_train})
#             for g, v in gradients_and_vars:
#                 if g is not None:
#                     print "****************this is variable*************"
#                     print "variable's shape:", v.shape
#                     print v
#                     print "****************this is gradient*************"
#                     print "gradient's shape:", g.shape
#                     print g
            #print(w1,w2)
            #assign_op = W1.assign(tf.add(W1,tf.multiply(tf.random_normal([input_vector_length,hidden_layer_nodes],stddev=0.1),tf.math.pow(alpha,tf.constant(i,'float32')))))
            
            average_loss = c/total_batch
            print(sess.run(mse, feed_dict={Input_layer:y_validate , Output_vectors: x_validate}))
        print("Epoch:", (epoch + 1), "cost =", "{:.3f}".format(average_loss))



1
0.03484469
('Epoch:', 1, 'cost =', '1.585')
0.035054125
('Epoch:', 2, 'cost =', '1.573')
0.03528189
('Epoch:', 3, 'cost =', '1.561')
0.035527986
('Epoch:', 4, 'cost =', '1.548')
0.035792347
('Epoch:', 5, 'cost =', '1.536')
0.036074955
('Epoch:', 6, 'cost =', '1.524')
0.036375694
('Epoch:', 7, 'cost =', '1.512')
0.036694482
('Epoch:', 8, 'cost =', '1.500')
0.037031215
('Epoch:', 9, 'cost =', '1.488')
0.03738577
('Epoch:', 10, 'cost =', '1.476')
0.037757963
('Epoch:', 11, 'cost =', '1.465')
0.03814765
('Epoch:', 12, 'cost =', '1.453')
0.038554613
('Epoch:', 13, 'cost =', '1.442')
0.038978647
('Epoch:', 14, 'cost =', '1.430')
0.03941951
('Epoch:', 15, 'cost =', '1.419')
0.03987693
('Epoch:', 16, 'cost =', '1.407')
0.040350627
('Epoch:', 17, 'cost =', '1.396')
0.040840283
('Epoch:', 18, 'cost =', '1.385')
0.04134556
('Epoch:', 19, 'cost =', '1.374')
0.0418661
('Epoch:', 20, 'cost =', '1.363')
0.042401515
('Epoch:', 21, 'cost =', '1.352')
0.042951405
('Epoch:', 22, 'cost =', '1.341')
0.04

0.22002189
('Epoch:', 181, 'cost =', '0.067')
0.22112502
('Epoch:', 182, 'cost =', '0.064')
0.22221944
('Epoch:', 183, 'cost =', '0.061')
0.22330494
('Epoch:', 184, 'cost =', '0.059')
0.22438124
('Epoch:', 185, 'cost =', '0.056')
0.22544815
('Epoch:', 186, 'cost =', '0.054')
0.2265054
('Epoch:', 187, 'cost =', '0.052')
0.22755277
('Epoch:', 188, 'cost =', '0.049')
0.22858998
('Epoch:', 189, 'cost =', '0.047')
0.22961697
('Epoch:', 190, 'cost =', '0.045')
0.23063338
('Epoch:', 191, 'cost =', '0.043')
0.23163913
('Epoch:', 192, 'cost =', '0.041')
0.23263384
('Epoch:', 193, 'cost =', '0.039')
0.23361742
('Epoch:', 194, 'cost =', '0.037')
0.2345896
('Epoch:', 195, 'cost =', '0.036')
0.23555034
('Epoch:', 196, 'cost =', '0.034')
0.23649926
('Epoch:', 197, 'cost =', '0.032')
0.23743637
('Epoch:', 198, 'cost =', '0.031')
0.23836145
('Epoch:', 199, 'cost =', '0.029')
0.2392743
('Epoch:', 200, 'cost =', '0.028')
0.24017477
('Epoch:', 201, 'cost =', '0.026')
0.2410628
('Epoch:', 202, 'cost =', '

In [19]:
print(w1)

[[-0.4028427  -0.2739633 ]
 [ 0.06689591  0.19194311]]


In [18]:
print(np.linalg.inv(A))

[[-0.10352908  0.06646819]
 [ 0.22330459 -0.07716934]]
